# 일급 함수

일급 객체의 정의
- 런타임에 생성할 수 있다.
- 데이터 구조체의 변수나 요소에 할당할 수 있다.
- 함수 인수로 전달할 수 있다.
- 함수 결과로 반환할 수 있다.

## higher-order function
함수를 인수로 받거나, 함수를 결과로 반환하는 함수를 고위 함수라고 한다.

In [1]:
# 단어 리스트를 철자 역순으로 정렬하기
def reverse(word):
    return word[::-1]
print(reverse('testing'))

gnitset


## map filter reduce의 대안

`map()`과 `filter()`함수는 여전히 파이썬 3에 내장되어 있지만, **지능형 리스트와 제너레이터 표현식이 소개된 후에는 이 함수의 중요성이 떨어졌다.**
지능형 리스트나 제너레이터 표현식이 `map()`과 `filter()`의 조합이 처리하는 작업을 표현할 수 있을 뿐만 아니라 가독성도 더 좋기 때문이다.

In [8]:
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

fact = factorial

# map
print(list(map(fact, range(6))))
# 지능형 리스트
print([fact(n) for n in range(6)])
# filter
print(list(map(factorial, filter(lambda n: n % 2, range(6)))))
# 지능형 리스트
print([factorial(n) for n in range(6) if n % 2])

from functools import reduce
from operator import add
# reduce
reduce(add, range(100))
# sum
sum(range(100))

[1, 1, 2, 6, 24, 120]
[1, 1, 2, 6, 24, 120]
[1, 6, 120]
[1, 6, 120]


4950

In [9]:
fruits = ['strawberry', 'fig', 'apple']
sorted(fruits, key=lambda word: word[::-1])

['apple', 'fig', 'strawberry']

## 일곱 가지 맛의 콜러블 객체
호출 연산자인 `()`는  사용자 정의 함수 이외의 다른 객체에서도 적용할 수 있다.
호출할 수 있는 객체인지 알아보려면 `callable()` 내장 함수를 사용한다.
파이썬 데이터 모델 문서는 다음 일곱 가지 콜러블을 나열하고 있다.

- 사용자 정의 함수
    - def문이나 람다 표현식으로 생성한다.
- 내장 함수
    - len()이나 time.strftime()처럼 c언어로 구현된 함수(CPython의 경우)
- 내장 메서드
    - dict.get()처럼 C언어로 구현된 메서드
- 메서드
    - 클래스 본체에 정의된 함수
- 클래스
    - 호출될 때 클래스는 자신의 `__new__()` 메서드를 실행해서 객체를 생성하고, `__init__()`으로 초기화한 후, 최종적으로 호출자에 객체를 반환한다.파이썬에는 new 연산자가 없으므로 클래스를 호출하는 것은 함수를 호출하는 것과 동일하다.
- 클래스 객체
    - 클래스가 `__call__()` 메서드를 구현하면 이 클래스의 객체는 함수로 호출될 수 있다.
- 제너레이터 함수
    - `yield` 키워드를 사용하는 함수나 메서드, 이 함수가 호출되면 제너레이터 객체를 반환한다.

파이썬에는 다양한 콜러블형이 존재하므로 callabe() 내장함수를 사용해서 호출할 수 있는 객체인지 판단하는 방법이 가장 안전하다.

In [11]:
abs, str, 13

[callable(obj) for obj in (abs, str, 13)]

[True, True, False]

## 위치 매개변수에서 키워드 전용 매개변수까지

키워드 전용 인수를 이용해서 향상된, 파이썬 3의 지극히 융통성 있는 매개변수 처리 매커니즘은 파이썬 함수에서 볼 수 있는 가장 훌륭한 기능 중 하나이다.
함수를 호출할 때 반복 기능 객체나 매핑형을 별도의 인수로 폴발 시키는 *와 ** 기호도 이 메커니즘과 밀접하게 연관되어 있다.

In [19]:
# HTML을 생성하는 tag()함수. class는 파이썬에 정의된 키워드로, 이를 피해 class 속성을 전달하기 위해 키워드 전용 클래스 매개변수로 cls를 사용했다.
def tag(name, *content, cls=None, **attrs):
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(' %s="%s"' % (attr, value)
                           for attr, value
                           in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return  '\n'.join('<%s%s>%s</%s>' %
                          (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)

tag('br')
tag('p', 'hello')

'<p>hello</p>'

## 함수 애너테이션
파이썬 3는 함수의 매개변수와 반환값에 메타데이터를 추가할 수 있는 구문을 제공한다.

에너테이션을 전혀 처리하지 않으며, 단지 함수 객체 안에 dict형 __annotations__속성에 저장될 뿐이다.
애너테이션은 도구, 프레임워크, 데커레이터가 사용할 수 있는 메타데이터일 뿐이다.

In [3]:
def clip(text:str, max_len:'int > 0'=80) -> str:
    """
    max_len 앞이나 뒤의 마지막 공백에서 잘라낸 텍스트를 반환한다.
    :param text:
    :param max_len:
    :return:
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:
        end = len(text)
    return text[:end].rstrip()

In [7]:
# 함수 시그니처에서 애너테이션 추출하기
from inspect import signature
sig = signature(clip)
print(sig.return_annotation)
print(sig.parameters)

<class 'str'>
OrderedDict([('text', <Parameter "text: str">), ('max_len', <Parameter "max_len: 'int > 0' = 80">)])


## 함수형 프로그래밍을 위한 패키지
- operator
- functools


In [ ]:
from functools import reduce
from operator import mul

# 람다 표현
def fact(n):
    return reduce(lambda a, b: a*b, range(1, n+1))

# operator.mul
def fact(n):
    return reduce(mul, range(1, n+1))


### 튜플 리스트 처리하기

In [14]:
# 튜플의 리스트를 정렬하기 위한 itemgetter() 사용

metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

from operator import itemgetter
[city for city in sorted(metro_data, key=itemgetter(1))]

[('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
 ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
 ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
 ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
 ('New York-Newark', 'US', 20.104, (40.808611, -74.020386))]

### 객체 속성 처리